# US Travels : Business / Pleasure / Education / All
### Data Engineering Capstone Project ( Spark )

Note to self : Remove all count operations when copied to python programs as its lazy

<div style="background-color:#ffe5e5;">
<br>    
    <b>Important</b> : No need to execute this as it will overwrite <code>./outputs/</code> &amp; <code>./outputs-gzip/dfs_ids1.gzip</code>
<br>
<br>
</div>

##### Import packages

In [2]:
import sys
# Below used by fuzzy matching
!{sys.executable} -m pip install fuzzywuzzy[speedup]

In [3]:
import pandas as pd
import os
import glob
#import psycopg2
import pandas as pd
import numpy as np
#from sql_queries import *
import datetime as dt
import json
from fuzzywuzzy import fuzz, process

In [4]:
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType, FloatType, TimestampType, LongType, DateType, NullType

import datetime #Required for ts conversion
#from pyspark.sql.functions import udf
#from pyspark.sql import functions as F

#import pyspark.sql.functions as F

#from pyspark.sql.functions import lit
from pyspark.sql.functions import year, month, dayofmonth, hour, minute, second

#from pyspark.sql.functions import *
from pyspark.sql.functions import udf, lit, datediff, when, col

##### Create spark session with hadoop-aws package

In [5]:
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

In [6]:
spark.sparkContext.getConf().getAll()

[('spark.driver.host', '164d3a7415ec'),
 ('spark.app.id', 'local-1581749391129'),
 ('spark.submit.pyFiles',
  '/root/.ivy2/jars/saurfang_spark-sas7bdat-2.0.0-s_2.11.jar,/root/.ivy2/jars/com.epam_parso-2.0.8.jar,/root/.ivy2/jars/org.apache.logging.log4j_log4j-api-scala_2.11-2.7.jar,/root/.ivy2/jars/org.slf4j_slf4j-api-1.7.5.jar,/root/.ivy2/jars/org.scala-lang_scala-reflect-2.11.8.jar'),
 ('spark.repl.local.jars',
  'file:///root/.ivy2/jars/saurfang_spark-sas7bdat-2.0.0-s_2.11.jar,file:///root/.ivy2/jars/com.epam_parso-2.0.8.jar,file:///root/.ivy2/jars/org.apache.logging.log4j_log4j-api-scala_2.11-2.7.jar,file:///root/.ivy2/jars/org.slf4j_slf4j-api-1.7.5.jar,file:///root/.ivy2/jars/org.scala-lang_scala-reflect-2.11.8.jar'),
 ('spark.executor.id', 'driver'),
 ('spark.jars.packages', 'saurfang:spark-sas7bdat:2.0.0-s_2.11'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.files',
  'file:///root/.ivy2/jars/saurfang_spark-sas7bdat-2.0.0-s_2.11.jar,file:///root/.ivy2/jars/com.epam_parso-2.0.8.

In [7]:
spark

In [8]:
# Panda options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

# for all rows set to None
# pd.set_option('display.max_rows', None) 

pd.set_option('display.max_rows', 100) 

##### Below are some prerequisite before cleaning i94 Immigration

##### Function

In [9]:
def strip_all_columns(df):
    """
    Summary line.
    Strip all columns in a dataframe
    
    Parameters:
    arg1 (dataframe)
    
    Returns:
    dataframe
    """
    for colu in df.columns:
        if df[colu].dtype == 'object':            
            #print('COL = ',col,' - ',df[col].dtype)
            #df[col] = df[col].str.strip()           
            mask = df[colu].notnull()
            df.loc[mask, colu] = df.loc[mask, colu].map(str.strip)            
            #df[col] = df[col].map(str.strip)
    return df

def difference (list1, list2): 
    """
    Summary line. 
    Difference between two lists used to compare list of two sets of columns

    Parameters: 
    arg1 (list 1)
    arg2 (list 2)

    Returns: 
    list with difference
    
    Sample : print(difference(keep_columns, df_jan.columns))
    """                
    list_dif = [i for i in list1 + list2 if i not in list1 or i not in list2]
    return list_dif

def fuzzymatch_city_get_ratio(row):
    """
    Summary line.
    Match city with US Cities list
    
    Parameters:
    arg1 (dataframe row)
    
    Returns:
    Panda series containing matched city name & score
    """
    city, state_code = row['City'], row['State Code']
    #print('City = ', city, 'State code = ',state_code)
    cities = df_uszips.loc[df_uszips.state_id==state_code, 'city'].str.lower().unique()    
    #print(process.extractOne(city, cities, scorer=fuzz.ratio))
    return pd.Series(process.extractOne(city, cities, scorer=fuzz.ratio))


##### Gather

In [10]:
# JSON to dictionary
fname = 'inputs/i94cit.json'
i94cit = json.load(open(fname))
print(i94cit["582"])

fname = 'inputs/i94port.json'
i94port = json.load(open(fname))
print(i94port["HHW"])

MEXICO Air Sea, and Not Reported (I-94, no land arrivals)
HONOLULU, HI          


In [11]:
# Airport Code Table
df_ac = pd.read_csv('inputs/airport-codes_csv.csv')

# Convert JSON Dictionaries to Dataframes
df_i94countrycode = pd.DataFrame(list(i94cit.items()), columns=['code', 'country'])
print('df_i94countrycode = {}'.format(df_i94countrycode.code.value_counts().shape[0]))

df_USPoE = pd.DataFrame(list(i94port.items()), columns=['code', 'citystate'])
print('df_USPoE = {}'.format(df_USPoE.code.value_counts().shape[0]))

# International Airports
df_iap = pd.read_csv('inputs/InternationalAirports.csv', sep=',', encoding = "ISO-8859-1")

# Visa Type
df_visatype = pd.read_csv('inputs/visatype2.csv', sep='|')

# U.S. City Demographic Data
df_uscd = pd.read_csv('inputs/us-cities-demographics.csv', sep=';')

# World Temperature Data
df_temper = pd.read_csv('inputs/world_temperature.csv')

# Airliner Codes - Can be used join airline column in i94-immigration data
df_alc = pd.read_csv('inputs/airline-codes.csv')

# Can be used to map to df_ids.visapost column
df_visapost = pd.read_csv('inputs/visapost.csv')

# US zip-codes
df_uszips = pd.read_csv('inputs/uszips.csv')

# Country Codes
df_alpha2countrycode = pd.read_csv('inputs/country-codes2.csv', sep='|')

# World Cities
df_wc = pd.read_csv('inputs/worldcities.csv', sep=',')

# US States
df_USstatecode = pd.read_csv('inputs/us-states.csv', sep='|')

df_i94countrycode = 289
df_USPoE = 660


##### Cleaning

In [12]:
# Create copies of dataframe
df_visatype1 = df_visatype.copy()
df_ac1 = df_ac.copy()
df_USPoE1 = df_USPoE.copy()
df_uszips1 = df_uszips.copy()
df_i94countrycode1 = df_i94countrycode.copy()
df_uscd1 = df_uscd.copy()
df_temper1 = df_temper.copy()
df_wc1 = df_wc.copy()

# Running strip() on all string columns
df_i94countrycode1 =strip_all_columns(df_i94countrycode1).copy()
df_USPoE1 =strip_all_columns(df_USPoE1).copy()
df_temper1 = strip_all_columns(df_temper1).copy()
df_visatype1 = strip_all_columns(df_visatype1).copy()
df_wc1 = strip_all_columns(df_wc1).copy()

##### df_i94countrycode : Updates

In [13]:
# Update mexico
temp = ['MEXICO Air Sea, and Not Reported (I-94, no land arrivals)']
df_i94countrycode1.loc[df_i94countrycode1['country'].isin(temp), 'country']='MEXICO'
# Below replace was not working
#df_i94countrycode1['country'] = df_i94countrycode1['country'].str.replace('MEXICO Air Sea, and Not Reported (I-94, no land arrivals)','MEXICO')

# Remove invalid codes / Collapsed / No Country
df_i94countrycode1 = df_i94countrycode1[~df_i94countrycode1.country.str.lower().str.contains('invalid')]
df_i94countrycode1 = df_i94countrycode1[~df_i94countrycode1.country.str.lower().str.contains('no country')]
df_i94countrycode1 = df_i94countrycode1[~df_i94countrycode1.country.str.lower().str.contains('collapsed')]

##### df_USPoE : Updates

In [14]:
# df_USPoE : Split one column into two
df_USPoE1["citystate"] = df_USPoE1["citystate"].map(str.strip)

df_USPoE1[['city', 'state']] = df_USPoE1['citystate'].str.rsplit(",",n=1, expand=True)
df_USPoE1 = strip_all_columns(df_USPoE1)
# trim all columns
#for col in df_USPoE1.columns:
#    df_USPoE1[col] = df_USPoE1[col].str.strip()

# df_USPoE : Correct incorrect state code values
df_USPoE1 = strip_all_columns(df_USPoE1).copy()
#df_USPoE2['state'] = df_USPoE2['state'].str.replace('\(BPS\)', '')
df_USPoE1['state'] = df_USPoE1.state.str.replace(r'\(.*\)$', '')
df_USPoE1['state'] = df_USPoE1['state'].str.replace('#ARPT', '')
df_USPoE1['state'] = df_USPoE1['state'].str.replace('#INTL', '')
df_USPoE1['state'] = df_USPoE1['state'].str.replace('WASHINGTON', 'WA')
df_USPoE1['state'] = df_USPoE1['state'].str.replace('MX', 'MEXICO')

df_USPoE1['city'] = df_USPoE1['city'].str.replace('#ARPT', '')
df_USPoE1['city'] = df_USPoE1['city'].str.replace('-ARPT', '')
df_USPoE1['city'] = df_USPoE1['city'].str.replace('ARPT', '')
df_USPoE1['city'] = df_USPoE1['city'].str.replace('INTL', '')
df_USPoE1 = strip_all_columns(df_USPoE1).copy()

# df_USPoE : Remove invalid rows & ports outside US
df_USPoE1 = strip_all_columns(df_USPoE1).copy()
df_USPoE1 = df_USPoE1[df_USPoE1.state.notnull()]

cond1 = df_USPoE1.city.str.lower().str.contains('collapsed')
df_USPoE1 = df_USPoE1[~cond1]

cond1 = df_USPoE1.city.str.lower().str.contains('no port')
df_USPoE1 = df_USPoE1[~cond1]

cond1 = df_USPoE1.city.str.lower().str.contains('unknown')
df_USPoE1 = df_USPoE1[~cond1]

cond1 = df_USPoE1.city.str.lower().str.contains('identifi')
df_USPoE1 = df_USPoE1[~cond1]

df_USPoE1 = df_USPoE1[df_USPoE1.state.str.len() == 2]

##### df_ac : Updates

In [15]:
# df_ac : Keeping only US Airports which serves passenger flights and has customs & immigration
# Dropping unused columns gps_code, ident
drop_cols = ['gps_code', 'ident']
df_ac1.drop(drop_cols, inplace = True, axis = 1)

#Keep only US Airports
df_ac1 = df_ac1[df_ac1.iso_country =='US'].copy()

# Correcting wrong continent value 
cond1 = df_ac1.iso_country =='US'
cond4 = df_ac1['continent'].isnull()
ix = df_ac1[cond1 & ~cond4].index.values[0]
df_ac1.at[ix,'continent']=np.nan

# Eliminate rows which has iata code as null
df_ac1 = df_ac1[~df_ac1.iata_code.isnull()].copy()

# Keep only airport which is of type small_airport, medium_airport & large_airport
temp = ['small_airport', 'medium_airport', 'large_airport']
df_ac1 = df_ac1[df_ac1['type'].isin(temp)].copy()

# Split geo-coordinates into separate columns
df_ac1[['longitude','latitude']] = df_ac1.coordinates.str.split(", ",expand=True)

# Extract state code from iso_region
df_ac1['state'] = df_ac1.iso_region.str.slice(start=3)

# Add new column facilities to indicate whether airport has Customs & Immigration
for code in df_iap['IATA']:
    #print(code)    
    df_ac1.loc[df_ac1['iata_code'] == code, 'facilities'] = 'CI'

#Keeping only airports which has Customs & Immigration
df_ac1 = df_ac1[df_ac1.facilities.notnull()].copy()
    
# Removing rows having 'Duplicate' in airport names
df_ac1 = df_ac1[~df_ac1.name.str.contains('uplicate')]

##### df_ids : Updates

In [16]:
# Checking files in the directory
data_dir = "../../data/18-83510-I94-Data-2016/"
files = os.listdir(data_dir)
for i in range(len(files)):
    files[i] = data_dir + files[i] 
    df_temp = spark.read.format('com.github.saurfang.sas.spark').load(files[i])
    print('{}. {} - {}'.format(i, len(df_temp.columns), files[i]))
#files
#jun16 has 34 columns

0. 28 - ../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat
1. 28 - ../../data/18-83510-I94-Data-2016/i94_sep16_sub.sas7bdat
2. 28 - ../../data/18-83510-I94-Data-2016/i94_nov16_sub.sas7bdat
3. 28 - ../../data/18-83510-I94-Data-2016/i94_mar16_sub.sas7bdat
4. 34 - ../../data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat
5. 28 - ../../data/18-83510-I94-Data-2016/i94_aug16_sub.sas7bdat
6. 28 - ../../data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat
7. 28 - ../../data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat
8. 28 - ../../data/18-83510-I94-Data-2016/i94_oct16_sub.sas7bdat
9. 28 - ../../data/18-83510-I94-Data-2016/i94_jul16_sub.sas7bdat
10. 28 - ../../data/18-83510-I94-Data-2016/i94_feb16_sub.sas7bdat
11. 28 - ../../data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat


In [17]:
%%time
dfs_ids =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

CPU times: user 950 µs, sys: 238 µs, total: 1.19 ms
Wall time: 106 ms


##### Functions

In [18]:
def to_datetime(x):
    try:
        start = dt.datetime(1960, 1, 1).date()
        return start + dt.timedelta(days=int(x))
    except:
        return None
udf_to_datetime_sas = udf(lambda x: to_datetime(x), DateType())


def to_datetimefrstr(x):
    try:
        return dt.datetime.strptime(x, '%m%d%Y')
    except:
        return None

udf_to_datetimefrstr = udf(lambda x: to_datetimefrstr(x), DateType())

def cdf_Ymd_to_mmddYYYY(x):
    try:
        return dt.datetime.strptime(x, '%Y%m%d')
    except:
        return None

udf_cdf_Ymd_to_mmddYYYY = udf(lambda x: cdf_Ymd_to_mmddYYYY(x), DateType())

def cdf_mdY_to_mmddYYYY(x):
    try:
        return dt.datetime.strptime(x, '%m%d%Y')
    except:
        return None

udf_cdf_mdY_to_mmddYYYY = udf(lambda x: cdf_mdY_to_mmddYYYY(x), DateType())

In [19]:
dfs_ids1 = dfs_ids

##### Convert from pandas to spark

In [20]:
dfs_i94countrycode1 = spark.createDataFrame(df_i94countrycode1)
dfs_USPoE1 = spark.createDataFrame(df_USPoE1)
dfs_ac1 = spark.createDataFrame(df_ac1)

ac_schema = StructType([
    StructField("type", StringType()),
    StructField("name", StringType()),
    StructField("elevation_ft", FloatType()),
    StructField("continent", StringType()),
    StructField("iso_country", StringType()),
    StructField("iso_region", StringType()),
    StructField("muncipality", StringType()),
    StructField("iata_code", StringType()),
    StructField("local_code", StringType()),
    StructField("coordinates", StringType()),   
    StructField("longitude", StringType()),   
    StructField("latitude", StringType()),   
    StructField("state", StringType()),   
    StructField("facilities", StringType()),   
])
dfs_ac1 = spark.createDataFrame(df_ac1, ac_schema)

In [21]:
# matflag is null
dfs_ids1 = dfs_ids1.filter(dfs_ids1.matflag.isNotNull())

In [22]:
# visatype GMT  
temp = df_visatype1.visatype.tolist()
dfs_ids1 = dfs_ids1.filter( dfs_ids1.visatype.isin(temp) )

In [23]:
# i94mode other than 1 2 3
temp = [1, 2, 3]
dfs_ids1 = dfs_ids1.filter( dfs_ids1.i94mode.isin(temp) )

In [24]:
# gender is null
dfs_ids1 = dfs_ids1.filter(dfs_ids1.gender.isNotNull())

In [25]:
# Remove rows having invalid CoC & CoR
temp = df_i94countrycode1.code.astype('int').tolist()
dfs_ids1 = dfs_ids1.filter( dfs_ids1.i94cit.isin(temp) )
dfs_ids1 = dfs_ids1.filter( dfs_ids1.i94res.isin(temp) )

In [26]:
%%time
# Remove Non-US Port of entry data
df1 = dfs_USPoE1.select('code')
df2 = dfs_ids1.select('i94port')
temp = df2.subtract(df1) #.collect()
tempArr = [row.i94port for row in temp.collect()]
dfs_ids1 = dfs_ids1.filter(~dfs_ids1.i94port.isin(tempArr))

CPU times: user 85.8 ms, sys: 14.3 ms, total: 100 ms
Wall time: 1min 59s


In [27]:
temp = dfs_ac1.filter( dfs_ac1.facilities.isNotNull() ).select('state').dropDuplicates()
df1 = [row.state for row in temp.collect()]
print('Total US States that has Customs & Immigration : ',len(df1))

Total US States that has Customs & Immigration :  41


In [28]:
%%time
# This takes 2 mins due to count() without it takes 3.36s
# Delete Port of Entries which lacks International Passenger Handling Facility operated by US Customs.
# df_ac has to be processed after df_USPoE1 and before df_ids1
temp = dfs_ac1.filter( dfs_ac1.facilities.isNotNull() ).select('state').dropDuplicates()
df1 = [row.state for row in temp.collect()]
print('Total US States that has Customs & Immigration : ',len(df1))
temp = dfs_USPoE1.filter(~dfs_USPoE1.state.isin(df1)).select('code').dropDuplicates()
df2 = [row.code for row in temp.collect()]
print('Number of Port of Entries that needs to be deleted from df_USPoE : ',len(df2))
#print('Total rows that will be deleted from df_ids due to PoE that doesnt have Customs & Immigration : '
#      ,dfs_ids1.filter( dfs_ids1.i94port.isin(df2)).count())
#print('Before : dfs_ids1 = ',dfs_ids1.count())
dfs_ids1 = dfs_ids1.filter(~dfs_ids1.i94port.isin(df2))
#print('After  : dfs_ids1 = ',dfs_ids1.count())
#

Total US States that has Customs & Immigration :  41
Number of Port of Entries that needs to be deleted from df_USPoE :  105
CPU times: user 128 ms, sys: 6.24 ms, total: 134 ms
Wall time: 5.33 s


In [29]:
# Dropping unused columns
# Only below columns are used for analysis
keep_columns = ['i94cit', 'i94res', 'i94port', 'arrdate', 'i94mode', 'i94addr', 'depdate'
                , 'i94bir', 'i94visa', 'dtadfile', 'visapost', 'occup', 'biryear'
                , 'dtaddto', 'gender', 'airline', 'admnum', 'fltno', 'visatype']
drop_cols = difference(keep_columns, dfs_ids1.columns)
print(drop_cols)

['cicid', 'i94yr', 'i94mon', 'count', 'entdepa', 'entdepd', 'entdepu', 'matflag', 'insnum']


In [30]:
dfs_ids1 = dfs_ids1.drop(*drop_cols)

In [31]:
#data_df = data_df.withColumn("Plays", data_df["Plays"].cast(IntegerType()))
# Convert floats to ints
cols_to_convert_float_to_integer = ['i94cit', 'i94res', 'arrdate', 'i94mode', 'depdate', 'i94bir'
                                , 'i94visa', 'biryear', 'admnum']
for colu in cols_to_convert_float_to_integer:    
    dfs_ids1 = dfs_ids1.na.fill(0, subset=[colu])
    dfs_ids1 = dfs_ids1.withColumn(colu, dfs_ids1[colu].cast(IntegerType()))

In [32]:
# Mapping : Codes to descriptive
#temp = {'1' : 'Air', '2' : 'Sea', '3' : 'Land', '9' : 'Not reported'}
temp = [["1", "Air"], ["2", "Sea"], ["3","Land"], ["9", "Not reported"]]
i94mode = spark.sparkContext.parallelize(temp).toDF(["code", "arrival_mode"])
dfs_ids1 = dfs_ids1.join(i94mode, dfs_ids1.i94mode == i94mode.code).select(dfs_ids1["*"], i94mode["arrival_mode"])

temp = [["1", "Business"], ["2", "Pleasure"], ["3", "Student"]]
i94visa = spark.sparkContext.parallelize(temp).toDF(["code", "visit_purpose"])
dfs_ids1 = dfs_ids1.join(i94visa, dfs_ids1.i94visa == i94visa.code).select(dfs_ids1["*"], i94visa["visit_purpose"])

In [33]:
#from pyspark.sql.functions import *
# Conversion of SAS encoded dates(arrdate & depdate)
dfs_ids1 = dfs_ids1.withColumn("arrival_dt", udf_to_datetime_sas(dfs_ids1.arrdate))
dfs_ids1 = dfs_ids1.withColumn("departure_dt", udf_to_datetime_sas(dfs_ids1.depdate))

dfs_ids1 = dfs_ids1.withColumn("DaysinUS", datediff("departure_dt", "arrival_dt"))

dfs_ids1 = dfs_ids1.withColumn("added_to_i94", udf_cdf_Ymd_to_mmddYYYY(dfs_ids1.dtadfile))
dfs_ids1 = dfs_ids1.withColumn("allowed_until", udf_cdf_mdY_to_mmddYYYY(dfs_ids1.dtaddto))

# Below corrections are carried out due to above adding 1960-01-01
dfs_ids1 = dfs_ids1.withColumn("arrival_dt",when(col("arrival_dt")=="1960-01-01",lit(None)).otherwise(col("arrival_dt")))
dfs_ids1 = dfs_ids1.withColumn("departure_dt",when(col("departure_dt")=="1960-01-01",lit(None)).otherwise(col("departure_dt")))
dfs_ids1 = dfs_ids1.withColumn("DaysinUS",when(col("arrival_dt").isNull(),lit(None)).otherwise(col("DaysinUS")))
dfs_ids1 = dfs_ids1.withColumn("DaysinUS",when(col("departure_dt").isNull(),lit(None)).otherwise(col("DaysinUS")))

# Departure date can't before Arrival date 
# ~(arrival date > departure date ) or (departure date can be null)
dfs_ids1 = dfs_ids1.filter(~(dfs_ids1.arrival_dt > dfs_ids1.departure_dt) | (dfs_ids1.departure_dt.isNull()))

# Change date format to YYYY-mm-dd
dfs_ids1 = dfs_ids1.withColumn("added_to_i94", udf_cdf_Ymd_to_mmddYYYY(dfs_ids1.dtadfile))
dfs_ids1 = dfs_ids1.withColumn("allowed_until", udf_cdf_mdY_to_mmddYYYY(dfs_ids1.dtaddto))

In [34]:
print(col)
print(when)
print(lit)

<function _create_function.<locals>._ at 0x7f96a082b840>
<function when at 0x7f96a07dcea0>
<function _create_function.<locals>._ at 0x7f96a082b7b8>


In [35]:
# Columns Rename
dfs_ids1 = (dfs_ids1
            .withColumnRenamed("i94bir",  "age")
            .withColumnRenamed("i94cit", "CoC")
            .withColumnRenamed("i94res", "CoR")
            .withColumnRenamed("i94port", "PoE")
            .withColumnRenamed("i94addr", "landing_state")
            .withColumnRenamed("visapost", "visa_issued_in"))

In [36]:
# Entry_Exit
dfs_ids1 = dfs_ids1.withColumn("entry_exit",when(col("departure_dt").isNull(),lit("entry")).otherwise(lit("exit")))

In [37]:
#dfs_ids1.groupby(dfs_ids1.entry_exit).count().show()

In [38]:
# Gender X to O
dfs_ids1 = dfs_ids1.withColumn("gender", when(col("gender")=="X", lit("O")).otherwise(col("gender")))

In [39]:
# Final Drop of unused columns
drop_cols = ['i94mode', 'i94visa', 'arrdate', 'depdate', 'dtadfile', 'dtaddto']
#print(drop_cols)
dfs_ids1 = dfs_ids1.drop(*drop_cols)

In [40]:
# Adding month which is used when saving file in parquet format partioning by month & landing state
dfs_ids1 = dfs_ids1.withColumn("month", month("arrival_dt"))

In [52]:
#Lots of displays
#dfs_ids1.collect()

In [42]:
dfs_ids2 = dfs_ids1

In [43]:
dfs_ids1 = dfs_ids2

In [44]:
dfs_ids1.printSchema()

root
 |-- CoC: integer (nullable = true)
 |-- CoR: integer (nullable = true)
 |-- PoE: string (nullable = true)
 |-- landing_state: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- visa_issued_in: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: integer (nullable = true)
 |-- fltno: string (nullable = true)
 |-- visatype: string (nullable = true)
 |-- arrival_mode: string (nullable = true)
 |-- visit_purpose: string (nullable = true)
 |-- arrival_dt: date (nullable = true)
 |-- departure_dt: date (nullable = true)
 |-- DaysinUS: integer (nullable = true)
 |-- added_to_i94: date (nullable = true)
 |-- allowed_until: date (nullable = true)
 |-- entry_exit: string (nullable = false)
 |-- month: integer (nullable = true)



##### df_visatype : Strip space from code column

In [45]:
df_visatype1["visatype"] = df_visatype1["visatype"].map(str.strip)

##### df_uszips : drop unused columns (zcta, parent_zcta, all_county_weights, imprecise, military, timezone)

In [46]:
drop_cols = ['zcta', 'parent_zcta', 'all_county_weights', 'imprecise', 'military', 'timezone']
df_uszips1.drop(drop_cols, inplace = True, axis = 1)

##### df_uscd : Updates to demographic dataframe

In [47]:
df_uscd1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
City                      2891 non-null object
State                     2891 non-null object
Median Age                2891 non-null float64
Male Population           2888 non-null float64
Female Population         2888 non-null float64
Total Population          2891 non-null int64
Number of Veterans        2878 non-null float64
Foreign-born              2878 non-null float64
Average Household Size    2875 non-null float64
State Code                2891 non-null object
Race                      2891 non-null object
Count                     2891 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 271.1+ KB


In [48]:
# Convert floats to ints
cols_to_convert_float_to_int = ['Male Population', 'Female Population', 'Number of Veterans', 'Foreign-born', 'Total Population', 'Count']

for col in cols_to_convert_float_to_int:
    df_uscd1[col] = df_uscd1[col].replace(np.nan, 0)
    df_uscd1[col] = df_uscd1[col].astype(int)

# Keeping only rows where MP + FP = TP 
df_uscd1 = df_uscd1[(df_uscd1['Male Population'] + df_uscd1['Female Population'] == df_uscd1['Total Population'])].copy()
    
# US : Cities check to if any names are misspelled
df1 = df_uszips[['city']]
lst = df1.city.str.lower().unique()
test = df_uscd1[~df_uscd1.City.str.lower().isin(lst)]

# removing rows of misspelled cities from df_uscd1 to process separately
df_uscd1 = df_uscd1[df_uscd1.City.str.lower().isin(lst)]

test1 = test.copy()

# Performing fuzzy match against the city rows which were not found in df_uszips
test2 = test1.apply(fuzzymatch_city_get_ratio, axis=1)
test2.columns = ['new_city_name','score']
test1 = test1.join(test2)

# Updating city names with score >= 80
test1['City'] = test1.apply(lambda x: x['new_city_name'] if x.score >= 80 else x['City'], axis=1)

# Convert lowercase city names to title case as before
test1.City = test1.City.str.strip().str.title()

# Dropping columns in fuzzy match dataframes
drop_cols = ['new_city_name', 'score']
test1.drop(drop_cols, inplace = True, axis = 1)

# Concatinating all the dataframes 1
test3 = [df_uscd1, test1]
df_uscd1 = pd.concat(test3).copy()

# Merging duplicates by avging & summing
temp = df_uscd1[df_uscd1.City=='Los Angeles'].copy()
df_uscd1 = df_uscd1[df_uscd1.City!='Los Angeles'].copy()
temp = temp.groupby(['City','State', 'State Code', 'Race']).agg({'Median Age':'mean','Male Population':'sum', 'Female Population': 'sum'
                                                          , 'Number of Veterans':'sum', 'Foreign-born':'sum'
                                                          , 'Average Household Size':'mean', 'Count':'sum' }).reset_index().copy()
# Concatinating all the dataframes 2
frames = [df_uscd1, temp]
# Encountered this warning while doing pd.concat : FutureWarning: Sorting because non-concatenation axis is not aligned. A future version of pandas will change to not sort by default.
df_uscd1 = pd.concat(frames, sort=False).copy()

# Above concat process changes the total population to float, so converting it back to Int
df_uscd1['Total Population'] = df_uscd1['Total Population'].replace(np.nan, 0)
df_uscd1['Total Population'] = df_uscd1['Total Population'].astype(int)

# Split based on Gender
df_usdp = df_uscd1[['State', 'City', 'Median Age', 'State Code', 'Male Population', 'Female Population', 'Total Population', 'Number of Veterans', 'Foreign-born']]
df_usdp = df_usdp.drop_duplicates()

# Split based on Race
df_usdr = df_uscd1[['State', 'City', 'State Code', 'Race', 'Count']].copy()

# Summing up the duplicates(ie., East Los Angeles -> Los Angeles)
df_usdr = df_usdr.groupby(['State', 'City', 'State Code', 'Race']).sum().reset_index().copy()

# Unmelting to converts Race rows to columns
df_usdr = df_usdr.set_index(['State','City','State Code', 'Race']).Count.unstack().reset_index()
df_usdr.columns.name = None
df_usdr[df_usdr.State =='Texas'].head()

# Convert floats to ints
cols_to_convert_float_to_int = ['American Indian and Alaska Native','Asian', 'Black or African-American', 'Hispanic or Latino', 'White']

for col in cols_to_convert_float_to_int:
    df_usdr[col] = df_usdr[col].replace(np.nan, 0)
    df_usdr[col] = df_usdr[col].astype(int)
    
# Rename column names
df_usdr.rename(columns={"State Code":"State_Code", "American Indian and Alaska Native": "American_Indian_and_Alaska_Native"
                        , "Black or African-American":"Black_or_African_American"
                        , "Hispanic or Latino":"Hispanic_or_Latino"}, inplace=True)

df_usdp.rename(columns={ "Median Age": "Median_Age" , "State Code":"State_Code", "Male_Population":"Male_Population"
                         , "Female Population":"Female_Population", "Total Population":"Total_Population" 
                         , "Number of Veterans":"Number_of_Veterans", "Foreign-born" : "Foreign_born" }, inplace=True)

##### df_temp : Updates to weather data

In [49]:
# Keeping only USA
df_temper1 = df_temper1[df_temper1.Country=='United States'].copy()

# Rounding temperature to decimals=3 
df_temper1['AverageTemperature'] = df_temper1['AverageTemperature'].round(decimals=3)

# Dropping unused columns
drop_cols = ['AverageTemperatureUncertainty', 'Latitude', 'Longitude']
df_temper1.drop(drop_cols, inplace = True, axis = 1)

# Removing missing temperatures
df_temper1 = df_temper1[~(df_temper1.AverageTemperature.isnull())]

# Eliminating the duplicates(ie., multiple locations in same city)
df_temper1 = df_temper1.drop_duplicates(['dt', 'City', 'Country'],keep= 'first')

# Convert dt to datetime from object
df_temper1['dt'] = pd.to_datetime(df_temper1.dt)

##### df_wc : Updates to world cities data

In [50]:
df_wc1['population'] = df_wc1['population'].replace(np.nan, 0)
df_wc1['population'] = df_wc1['population'].astype(int)

#### Store
##### Save clean dataframe to disk

In [50]:
print("Cleaned datasets")
#print("{} = {}".format("df_ids1", df_ids1.shape[0]))
print("{} = {}".format("df_visatype1", df_visatype1.shape[0]))
print("{} = {}".format("df_ac1", df_ac1.shape[0]))
print("{} = {}".format("df_USPoE1", df_USPoE1.shape[0]))
print("{} = {}".format("df_uszips1", df_uszips1.shape[0]))
print("{} = {}".format("df_i94countrycode1", df_i94countrycode1.shape[0]))
print("{} = {}".format("df_uscd1", df_uscd1.shape[0]))
print("{} = {}".format("df_usdp", df_usdp.shape[0]))
print("{} = {}".format("df_usdr", df_usdr.shape[0]))
print("{} = {}".format("df_temper1", df_temper1.shape[0]))
print("{} = {}".format("df_wc1", df_wc1.shape[0]))

print("Datasets with no changes/updates")
print("{} = {}".format("df_alc", df_alc.shape[0]))
print("{} = {}".format("df_visapost", df_visapost.shape[0]))
print("{} = {}".format("df_alpha2countrycode", df_alpha2countrycode.shape[0]))
print("{} = {}".format("df_iap", df_iap.shape[0]))
print("{} = {}".format("df_USstatecode", df_USstatecode.shape[0]))

Cleaned datasets
df_visatype1 = 108
df_ac1 = 113
df_USPoE1 = 526
df_uszips1 = 33099
df_i94countrycode1 = 236
df_uscd1 = 2883
df_usdp = 594
df_usdr = 594
df_temper1 = 639649
df_wc1 = 15493
Datasets with no changes/updates
df_alc = 1571
df_visapost = 284
df_alpha2countrycode = 249
df_iap = 1304
df_USstatecode = 62


In [51]:
# Cleaned datasets
#df_ids1.to_csv('outputs/df_ids1.csv', encoding='utf-8', index=False)
#dfs_ids1.write.csv('outputs/dfs_ids1_i94_apr16_sub.csv')
df_visatype1.to_csv('outputs/df_visatype1.csv', encoding='utf-8', index=False)
df_ac1.to_csv('outputs/df_ac1.csv', encoding='utf-8', index=False)
df_USPoE1.to_csv('outputs/df_USPoE1.csv', encoding='utf-8', index=False)
df_uszips1.to_csv('outputs/df_uszips1.csv', encoding='utf-8', index=False)
df_i94countrycode1.to_csv('outputs/df_i94countrycode1.csv', encoding='utf-8', index=False)
df_uscd1.to_csv('outputs/df_uscd1.csv', encoding='utf-8', index=False)
df_usdp.to_csv('outputs/df_usdp.csv', encoding='utf-8', index=False)
df_usdr.to_csv('outputs/df_usdr.csv', encoding='utf-8', index=False)
df_temper1.to_csv('outputs/df_temper1.csv', encoding='utf-8', index=False)
df_wc1.to_csv('outputs/df_wc1.csv', encoding='utf-8', index=False)

# Below datasets have not changes/updates, just writing to another folder
df_alc.to_csv('outputs/df_alc.csv', encoding='utf-8', index=False)
df_visapost.to_csv('outputs/df_visapost.csv', encoding='utf-8', index=False)
df_alpha2countrycode.to_csv('outputs/df_alpha2countrycode.csv', encoding='utf-8', index=False)
df_iap.to_csv('outputs/df_iap.csv', encoding='utf-8', index=False)
df_USstatecode.to_csv('outputs/df_USstatecode.csv', encoding='utf-8', index=False)

In [ ]:
#Successfully creates partitioned parquet - But i am getting errors while loading into Redshift from S3
#dfs_ids1.write.parquet('./outputs-parquet/i94-apr16.parquet', mode='overwrite', partitionBy=['month', 'landing_state'])    

#Parquet format output doesn't have null value fields due to which not able to load into redshift
#dfs_ids1.write.parquet('./outputs-parquet/i94-apr16.parquet', mode='overwrite')    

In [1]:
def get_aws_keysecret(filepath):
    with open(filepath) as myfile:
        head = [next(myfile) for x in range(4)]
    #print(head)
    #print(head[3])
    for s in head:
        #print("KEYSECRET" in s)
        if "KEYSECRET" in s:
            arr = s.split('KEYSECRET=')
            key = arr[1]
            #print(key)
            break
    return key.strip()

awskey = get_aws_keysecret("./aws/aws-capstone.cfg")
#print(awskey)

In [2]:
def make_copy_statement(table, data, awskey):
    copy_stmt = ("""
        COPY {} FROM {}
        CREDENTIALS '{}'
        TRUNCATECOLUMNS BLANKSASNULL EMPTYASNULL
        CSV 
        delimiter ',' 
        IGNOREHEADER 1
        COMPUPDATE OFF REGION 'us-west-2';
        """).format(table, data, awskey)
    return copy_stmt

awskey = get_aws_keysecret("./aws/aws-capstone.cfg")
#print(make_copy_statement('staging_ids', 's3://asdf', awskey))
a = {
    "dfs_ids1":"ds_ids1.csv",
    "dfs_ids":""
}

In [57]:
fname = 'inputs/staging-table-data.txt'
table_data = json.load(open(fname))
#print(table_data["staging_uspoe"])

for table, data in table_data.items():
    print(table, data)

staging_visatype 's3://sushanth-dend-capstone-files/outputs/df_visatype1.csv'
staging_ac 's3://sushanth-dend-capstone-files/outputs/df_ac1.csv'
staging_USPoE 's3://sushanth-dend-capstone-files/outputs/df_USPoE1.csv'
staging_uszips 's3://sushanth-dend-capstone-files/outputs/df_uszips1.csv'
staging_i94countrycode 's3://sushanth-dend-capstone-files/outputs/df_i94countrycode1.csv'
staging_usdp 's3://sushanth-dend-capstone-files/outputs/df_usdp.csv'
staging_usdr 's3://sushanth-dend-capstone-files/outputs/df_usdr.csv'
staging_temper 's3://sushanth-dend-capstone-files/outputs/df_temper1.csv'
staging_alc 's3://sushanth-dend-capstone-files/outputs/df_alc.csv'
staging_visapost 's3://sushanth-dend-capstone-files/outputs/df_visapost.csv'
staging_alpha2countrycode 's3://sushanth-dend-capstone-files/outputs/df_alpha2countrycode.csv'
staging_iap 's3://sushanth-dend-capstone-files/outputs/df_iap.csv'
staging_wc 's3://sushanth-dend-capstone-files/outputs/df_wc1.csv'
staging_USstatecode 's3://sushanth-d

In [59]:
from shutil import copyfile
copyfile('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat', './sas_inputs/i94_apr16_sub.sas7bdat')

'./sas_inputs/i94_apr16_sub.sas7bdat'

In [61]:
import datetime
from pyspark.sql.functions import year, month, dayofmonth

elevDF = spark.sparkContext.parallelize([
    (datetime.datetime(1984, 1, 1, 0, 0), 1, 638.55),
    (datetime.datetime(1984, 1, 1, 0, 0), 2, 638.55),
    (datetime.datetime(1984, 1, 1, 0, 0), 3, 638.55),
    (datetime.datetime(1984, 1, 1, 0, 0), 4, 638.55),
    (datetime.datetime(1984, 1, 1, 0, 0), 5, 638.55)
]).toDF(["date", "hour", "value"])

elevDF.select(
    year("date").alias('year'), 
    month("date").alias('month'), 
    dayofmonth("date").alias('day')
).show()


#temp = dfs_ac1.filter( dfs_ac1.facilities.isNotNull() ).select('state').dropDuplicates()
elevDF.select(
    month("date").alias('month')
    ).dropDuplicates().show()

+----+-----+---+
|year|month|day|
+----+-----+---+
|1984|    1|  1|
|1984|    1|  1|
|1984|    1|  1|
|1984|    1|  1|
|1984|    1|  1|
+----+-----+---+

+-----+
|month|
+-----+
|    1|
+-----+



In [62]:
dfs_ids1.printSchema()

root
 |-- CoC: integer (nullable = true)
 |-- CoR: integer (nullable = true)
 |-- PoE: string (nullable = true)
 |-- landing_state: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- visa_issued_in: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: integer (nullable = true)
 |-- fltno: string (nullable = true)
 |-- visatype: string (nullable = true)
 |-- arrival_mode: string (nullable = true)
 |-- visit_purpose: string (nullable = true)
 |-- arrival_dt: date (nullable = true)
 |-- departure_dt: date (nullable = true)
 |-- DaysinUS: integer (nullable = true)
 |-- added_to_i94: date (nullable = true)
 |-- allowed_until: date (nullable = true)
 |-- entry_exit: string (nullable = false)
 |-- month: integer (nullable = true)



In [64]:
dfs_ids1.columns

['CoC',
 'CoR',
 'PoE',
 'landing_state',
 'age',
 'visa_issued_in',
 'occup',
 'biryear',
 'gender',
 'airline',
 'admnum',
 'fltno',
 'visatype',
 'arrival_mode',
 'visit_purpose',
 'arrival_dt',
 'departure_dt',
 'DaysinUS',
 'added_to_i94',
 'allowed_until',
 'entry_exit',
 'month']

In [66]:
dfs_ids1.count()

2093913

In [69]:
dfs_ids1.write\
  .format("com.databricks.spark.csv")\
  .option("header","true")
  .option("codec", "org.apache.hadoop.io.compress.GzipCodec")\
  .save('./outputs-gzip/dfs_ids1.gzip')

In [11]:
# Checking files in the directory
data_dir = "../../data/18-83510-I94-Data-2016/"
files = os.listdir(data_dir)
for i in range(len(files)):
    files[i] = data_dir + files[i] 
    df_temp = spark.read.format('com.github.saurfang.sas.spark').load(files[i])
    file = files[i].split('/')
    print('{}. {} - {} : {}'.format(i, len(df_temp.columns), files[i], './sas_inputs/'+file[-1]))
#files
#jun16 has 34 columns

0. 28 - ../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat : ./sas_inputs/i94_apr16_sub.sas7bdat
1. 28 - ../../data/18-83510-I94-Data-2016/i94_sep16_sub.sas7bdat : ./sas_inputs/i94_sep16_sub.sas7bdat
2. 28 - ../../data/18-83510-I94-Data-2016/i94_nov16_sub.sas7bdat : ./sas_inputs/i94_nov16_sub.sas7bdat
3. 28 - ../../data/18-83510-I94-Data-2016/i94_mar16_sub.sas7bdat : ./sas_inputs/i94_mar16_sub.sas7bdat
4. 34 - ../../data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat : ./sas_inputs/i94_jun16_sub.sas7bdat
5. 28 - ../../data/18-83510-I94-Data-2016/i94_aug16_sub.sas7bdat : ./sas_inputs/i94_aug16_sub.sas7bdat
6. 28 - ../../data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat : ./sas_inputs/i94_may16_sub.sas7bdat
7. 28 - ../../data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat : ./sas_inputs/i94_jan16_sub.sas7bdat
8. 28 - ../../data/18-83510-I94-Data-2016/i94_oct16_sub.sas7bdat : ./sas_inputs/i94_oct16_sub.sas7bdat
9. 28 - ../../data/18-83510-I94-Data-2016/i94_jul16_sub.sas7bdat : ./sas_

In [12]:
# Checking files in the directory
data_dir = "../../data/18-83510-I94-Data-2016/"
files = os.listdir(data_dir)
sas_files = [data_dir + files[i] for i in range(len(files))]    
print(sas_files)

['../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat', '../../data/18-83510-I94-Data-2016/i94_sep16_sub.sas7bdat', '../../data/18-83510-I94-Data-2016/i94_nov16_sub.sas7bdat', '../../data/18-83510-I94-Data-2016/i94_mar16_sub.sas7bdat', '../../data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat', '../../data/18-83510-I94-Data-2016/i94_aug16_sub.sas7bdat', '../../data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat', '../../data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat', '../../data/18-83510-I94-Data-2016/i94_oct16_sub.sas7bdat', '../../data/18-83510-I94-Data-2016/i94_jul16_sub.sas7bdat', '../../data/18-83510-I94-Data-2016/i94_feb16_sub.sas7bdat', '../../data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat']


In [51]:
dfs_ids1.printSchema()

root
 |-- CoC: integer (nullable = true)
 |-- CoR: integer (nullable = true)
 |-- PoE: string (nullable = true)
 |-- landing_state: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- visa_issued_in: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: integer (nullable = true)
 |-- fltno: string (nullable = true)
 |-- visatype: string (nullable = true)
 |-- arrival_mode: string (nullable = true)
 |-- visit_purpose: string (nullable = true)
 |-- arrival_dt: date (nullable = true)
 |-- departure_dt: date (nullable = true)
 |-- DaysinUS: integer (nullable = true)
 |-- added_to_i94: date (nullable = true)
 |-- allowed_until: date (nullable = true)
 |-- entry_exit: string (nullable = false)
 |-- month: integer (nullable = true)

